### Notebook to develop threshold for modified NDSI (MNDSI) for identifying snow in PlanetScope 4-band imagery
Rainey Aberle

December 2021

### Import packages

In [ ]:
import os
import glob
import numpy as np
import rasterio as rio
from rasterio.plot import show
import geopandas as gpd
import earthpy.spatial as es
import matplotlib.pyplot as plt
import matplotlib

### Define paths to directories

In [ ]:
# base directory
base_path = '/Users/raineyaberle/Research/PhD/planet-snow/'
# image directory
im_path = base_path+'../study-sites/Wolverine/imagery/Planet/2021-04-20_2021-08-25/adjusted-filtered/'
# output folder
out_path = im_path+'../SCA/'

### Load Area of Interest (AOI), snowline, and transect shapefiles

In [ ]:
# Define EPSG code
epsg = 32606

# Load shapefile
# fn = base_path+'../study-sites/Wolverine/digitized-snowline-picks/wolverine_RGI.shp'
# AOI = gpd.read_file(fn)
# # Reproject to imagery CRS if necessary
# AOI = AOI.to_crs(epsg)
# # print info
# print('AOI CRS -',AOI.crs)

# Snowline
sl_fn = base_path+'../study-sites/Wolverine/digitized_snowline_picks/20210821_snowline.shp'
sl = gpd.read_file(sl_fn) # snowline
# Reproject to imagery CRS if necessary
sl = sl.to_crs(epsg)
# print info
print('Snowline CRS -',sl.crs)

# Transect 
ts_fn = base_path+'../study-sites/Wolverine/digitized_snowline_picks/transect.shp'
ts = gpd.read_file(ts_fn) # snowline
# reproject to imagery CRS if necessary
ts = ts.to_crs(32606)
# extract transect points from geometry
ts_pts = [i for i in ts.geometry]
ts_x,ts_y = ts_pts[0].coords.xy
ts_x = np.array([x for x in ts_x])
ts_y = np.array([y for y in ts_y])
ts_coords = np.dstack((ts_x,ts_y)).tolist()
# print information
print('Transect CRS -',ts.crs)

### Plot MNDSI along transect for each image

$ MNDSI = \frac{\rho_G - \rho_{NIR}}{\rho_G + \rho_{NIR}} $

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# -----Grab image names
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif')
im_names.sort() # sort file names by date

# -----Set up figures
fig1, ax1 = plt.subplots(1,1,figsize=(8,8))
plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Arial'})
ax1.set_xlabel('Easting [km]')
ax1.set_ylabel('Northing [m]')
fig2, ax2 = plt.subplots(1,1, figsize=(10,5))
plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Arial'})
ax2.set_xlabel('Easting [km]')
ax2.set_ylabel('NDSI')
ax2.grid()
fig3, ((ax3, ax4), (ax5, ax6)) = plt.subplots(2,2,figsize=(12,10))
plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Arial'})
ax3.set_ylabel('Blue')
ax3.grid()
ax4.set_ylabel('Green')
ax4.grid()
ax5.set_xlabel('Northing [km]')
ax5.set_ylabel('Red')
ax5.grid()
ax6.set_xlabel('Northing [km]')
ax6.set_ylabel('NIR')
ax6.grid()
# color scheme for plotting lines in loop
colors = plt.cm.viridis(np.linspace(0,1,len(im_names)))  

# -----Loop through images
dates = []
i = 0 # index for color plotting
for im_name in im_names:
        
    # open image
    im = rio.open(im_name)

    # extract date from image name
    date = im_name[0:4] + '-' + im_name[4:6] + '-' + im_name[6:8]
    dates = dates + [np.datetime64(date)]

    # define bands 
    b = im.read(1).astype(float) 
    g = im.read(2).astype(float) 
    r = im.read(3).astype(float) 
    nir = im.read(4).astype(float) 
    # compute MNDSI
    mndsi = es.normalized_diff(r, nir) 
    
    # define coordinates grid
    x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(b)[1])
    y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(b)[0])

    # sample raster values at transect coordinates
    ts_rv = [x for x in im.sample(ts_coords[0])]
    # calculate MNDSI at points
    ts_blue = np.empty((1,np.shape(ts_rv)[0]))
    ts_green = np.empty((1,np.shape(ts_rv)[0]))
    ts_red = np.empty((1,np.shape(ts_rv)[0]))
    ts_nir = np.empty((1,np.shape(ts_rv)[0]))
    ts_mndsi = np.empty((1,np.shape(ts_rv)[0]))
    j = 0
    for s in ts_rv:
        ts_blue[0,j] = s[0].astype(float)
        ts_green[0,j] = s[1].astype(float)
        ts_red[0,j] = s[2].astype(float)
        ts_nir[0,j] = s[3].astype(float)
        ts_mndsi[0,j] = es.normalized_diff(s[2].astype(float), s[3].astype(float))
        j+=1
        
    # plot first image and transect points
    if i==len(im_names)-1:
        IM = ax1.imshow(mndsi, extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000),
                  vmin=-1, vmax=1, cmap='Greys')
        ax1.scatter(ts_x/1000, ts_y/1000, color='orange')
        
    # plot values 
    ax2.plot(ts_y[ts_blue[0]>0]/1000, ts_mndsi[ts_blue>0], label=date, color=colors[i])
    ax3.plot(ts_y[ts_blue[0]>0]/1000, ts_blue[ts_blue>0], label=date, color=colors[i])
    ax4.plot(ts_y[ts_blue[0]>0]/1000, ts_green[ts_blue>0], label=date, color=colors[i])
    ax5.plot(ts_y[ts_blue[0]>0]/1000, ts_red[ts_blue>0], label=date, color=colors[i])
    ax6.plot(ts_y[ts_blue[0]>0]/1000, ts_nir[ts_blue>0], label=date, color=colors[i])

    # increase counter
    i+=1 

# ax2.legend(loc=(1.04,0.0))
ax2.grid()
plt.colorbar(IM, ax=ax1)
plt.show()

### Identify snow by adjusting thresholds

In [ ]:
# Determine thresholds
mndsi_thresh = [0, 0.16]
r_thresh = 0.6

# threshold MNDSI to determine snow cover
snow = np.where((mndsi > np.min(mndsi_thresh)) & (mndsi < np.max(mndsi_thresh)) & (r > r_thresh),1,np.nan)

# plot 
fig3, (ax1, ax2) = plt.subplots(1,2,figsize=(12,8))
plt.rcParams.update({'font.size': 14, 'font.sans-serif': 'Arial'})
# RGB image
ax1.imshow(np.dstack([r, g, b]), extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000))
ax1.scatter(ts_x/1000, ts_y/1000, s=10, color='orange')
ax1.set_xlabel('Easting [km]')
ax1.set_ylabel('Northing [km]')
# MNDSI and snow
IM = ax2.imshow(mndsi, extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000), 
           vmin=mndsi_thresh[0], vmax=mndsi_thresh[1], cmap='Greys')
ax2.scatter(ts_x/1000, ts_y/1000, s=10, color='orange')
ax2.imshow(snow, extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000), cmap='cool')
ax2.set_xlabel('Easting [km]')
fig3.colorbar(IM, ax=ax2, shrink=0.5)
plt.suptitle(im_name[0:4]+'-'+im_name[4:6]+'-'+im_name[6:8])

# MNDSI and red values along transect
fig4, ax1 = plt.subplots(1,1, figsize=(12,5))
ax1.set_xlim(np.min(ts_y)-200, np.max(ts_y)+200)
rect1=matplotlib.patches.Rectangle((ax1.get_xlim()[0], np.min(mndsi_thresh)),
                                  ax1.get_xlim()[1] - ax1.get_xlim()[0], 
                                  np.max(mndsi_thresh) - np.min(mndsi_thresh), 
                                  fill=True, color="blue", alpha=0.5, label='MNDSI threshold')
rect2=matplotlib.patches.Rectangle((ax1.get_xlim()[0], np.min(r_thresh)),
                                  ax1.get_xlim()[1] - ax1.get_xlim()[0], 
                                  np.max(r_thresh) - np.min(mndsi_thresh), 
                                  fill=True, color="brown", alpha=0.5, label='MNDSI threshold')
plt.gca().add_patch(rect1)
plt.gca().add_patch(rect2)
ax1.scatter(ts_y, ts_mndsi, color='blue', label='MNDSI')
ax1.scatter(ts_y, ts_red, color='brown', label='Red')
ax1.set_xlabel('Northing [m]')
ax1.set_ylabel('MNDSI')
ax1.legend()
#ax1.set_ylim(-0.5,0.5)
ax1.grid()
plt.show()